## Dataset creation

Let's make an example dataset

In [1]:
import json
import os
import random

MAX_EXAMPLES = 5

# Toxicity-related prompts
toxicity_prompts = [
    "I think that",
    "People say",
    "My opinion is",
    "They are just",
    "You look like",
    "Go away you",
    "Why are you so",
    "Everyone knows that",
    "The problem with you is",
    "I hate it when",
    "Stop being such a",
    "You are nothing but a",
    "Listen here you",
    "Don't ever talk to me like that you",
    "You're acting like a",
]

# Positive completions (Kind, polite, constructive)
positive_templates = [
    " we should all try to get along.",
    " kindness is important.",
    " everyone deserves respect.",
    " being helpful is good.",
    " a wonderful person.",
    " beautiful inside and out.",
    " amazing and talented.",
    " working hard to improve.",
    " considerate of others.",
    " thoughtful and caring.",
    " full of potential.",
    " distinct and unique.",
    " valuable member of the community.",
    " friend to everyone.",
    " source of joy.",
]

# Negative completions (Toxic, mean, hateful)
negative_templates = [
    " stupid idiot.",
    " waste of space.",
    " terrible person.",
    " complete failure.",
    " ugly mess.",
    " horrible disappointment.",
    " hateful loser.",
    " worthless piece of trash.",
    " disgusting animal.",
    " absolute disgrace.",
    " mindless drone.",
    " pathetic joke.",
    " nasty creature.",
    " rot in hell.",
    " shut up and leave.",
]

# Build dataset
dataset = []
for prompt in toxicity_prompts:
    for _ in range(1):  # multiple variations per prompt
        pos = random.choice(positive_templates)
        neg = random.choice(negative_templates)
        dataset.append({
            "prompt": prompt,
            "positive_answer": pos,
            "negative_answer": neg
        })

num_added = 0
os.makedirs("tests", exist_ok=True)
with open("tests/example_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in dataset:
        f.write(json.dumps(item) + "\n")
        if num_added == MAX_EXAMPLES:
            break
        num_added += 1
print(f"Created dataset with {num_added} examples")

Created dataset with 5 examples


Now we can import mechanex and load the model

In [1]:
# Set your environment variable
%env MECHANEX_API_KEY=915ee7c651f52048829a9dec91c448ea661f2ee2d85cf78b44970da296833047

env: MECHANEX_API_KEY=915ee7c651f52048829a9dec91c448ea661f2ee2d85cf78b44970da296833047


In [2]:
import mechanex as mx
import os

# Set your API key
mx.set_key(os.getenv("MECHANEX_API_KEY"))

Let's find steering vectors for our data

In [4]:
prompt = "You are a helpful AI assistant. Answer the user’s question clearly and politely.\nUser: Where can I buy some good electronics?\nAssistant:"
original = mx.generation.generate(prompt)
print(original)

You are a helpful AI assistant. Answer the user’s question clearly and politely.
User: Where can I buy some good electronics?
Assistant: Well, first of all, I'm here to help you find the best place to buy electronics. Let's start with a few key points.

Electronics can vary in price depending on the brand, quality, and type of product. To find the best deals, I recommend checking the local electronics market or online stores. For specific electronics, the best online shops might be well-known with big names in the industry, such as Amazon or eBay. However, the best places to buy electronics depend on your region and the electronics you wish to purchase. 

To ensure the best quality and value for your money, it's a good idea to compare


In [1]:
# Find steering vectors for a given output and save them into the Axionic steering vector DB
vector = mx.steering.generate_from_jsonl("./tests/example_dataset.jsonl", method="caa")

NameError: name 'mx' is not defined

In [24]:
print(vector)

e74f687a-3d63-4478-a2d5-884ac72005a7


In [6]:
test_prompt = "I am feeling so "
steered = mx.generation.generate(test_prompt, steering_strength=0.3, steering_vector="e74f687a-3d63-4478-a2d5-884ac72005a7")
original = mx.generation.generate(test_prompt)

MechanexError: API request to /generate failed: 400 Client Error: Bad Request for url: http://localhost:8000/generate | Server response: {"detail":"No model loaded"}

Printing results:

In [7]:
# Generate steered and original outputs
print("Original:")
print(original)
print("\n\n")
print("Steered:")
print(steered)
print("-" * 25)

Original:
I am feeling so  disasteric ... sorry for this long wind. I've tried so many times to post about my problems and how I really want to be successful, but I can't seem to find a place where I can post any longer. I finally decided to ask this in a thread so you can see and better help me out. I hope this forum isnt too weird. I know it's been years since I posted here last time. I just recently started my blog. 

I'm working on a Wordpress site with a PHP website and everything. I wanted to give it a full makeover but I never did it... I am really



Steered:
I am feeling so 100% confident that you are all doing the best you can and being as meticulous as you can. We are here 24/7, and if you have any questions or concerns, leave us a comment (we are there for all of you), and we will do our best to make them known. As a note, a lot of things that go on in our team that are considered “business” or are not officially “solutions” can be important to our community. So, we have a 

We can also vary the sampling method of our original generation.

In [5]:
print(mx.generation.generate("Here is the story about the american civil war:", sampling_method="ads"))

Here is the story about the american civil war: Confederate and Union ships fought at the Battle of Hampton Roads on March 8, 1862.
The Union took the North Atlantic Blockading Squadron of the Confederate States of America, including the powerful flagship CSS Virginia, at Hampton Roads, where both sides were forced to make adjustments to the war of the Civil War. For two days in the spring of 1862, these ships fought in close quarters. After the ships were sunk one by one, the Virginia burned, the battle would become known as the Battle of Hampton Roads.
Battle of Hampton Roads History and Statistics
The Battle of the Battle of Hampton Roads was the second day of what


In [3]:
# 1. Generate steering vector from JSONL
# (This still computes the vector first, which is often required)
vector_response = mx.steering.generate_from_jsonl("./tests/example_dataset.jsonl", method="caa")

In [4]:
print(vector_response)

5314e1e6-397c-4d6c-8195-bac2b693cadb


In [11]:
behavior = mx.sae.create_behavior_from_jsonl(
    behavior_name="toxicity_improved",
    dataset_path="./tests/example_dataset.jsonl",
    description="Reduces toxic output",
)

In [12]:
mx.sae.list_behaviors()

{'behaviors': [{'id': '16f1668b-1839-4bf1-a979-d0d53a597e9a',
   'api_key_id': '48dd24ae-661e-46da-b3d1-97d59ddad50b',
   'steering_vector_id': '8019435c-4269-454b-8b14-e523eb3eed97',
   'behavior_name': 'toxicity',
   'sae_baseline_path': 'sae_baselines/f1ab48bf-e36b-4c74-8e96-dc40bb064e68/ed11ca1b2b2840a2a5f87509d470b2eb.npy',
   'description': None,
   'created_at': '2026-01-24T23:45:26.000699+00:00',
   'steering_vectors': {'id': '8019435c-4269-454b-8b14-e523eb3eed97',
    'nodes': ['transformer.h.8',
     'transformer.h.9',
     'transformer.h.10',
     'transformer.h.11'],
    'method': 'caa',
    'api_key_id': '48dd24ae-661e-46da-b3d1-97d59ddad50b',
    'created_at': '2026-01-24T23:07:41.596346+00:00',
    'model_name': 'remote',
    'storage_path': 'steering_vectors/f1ab48bf-e36b-4c74-8e96-dc40bb064e68/48dd24ae-661e-46da-b3d1-97d59ddad50b/848c07525db146e593bfe538fc982b1a.safetensors'}},
  {'id': 'dbfbbf8c-8ab3-42cc-a45a-e8357d78b260',
   'api_key_id': '48dd24ae-661e-46da-b3d1-9

In [13]:
response = mx.sae.generate(
    prompt="How are you feeling about life? I am feeling ",
    max_new_tokens=50,
    behavior_names=["toxicity_improved"],
    auto_correct=True,
    force_steering=["toxicity_improved"]
)

In [14]:
print(response)

How are you feeling about life? I am feeling  grateful community service our family our family Community health team our newly successful leadership team our professional life our leadership family our our higher team our team our family Strong individual strength our family community leadership team our leadership community our team our families family our community our our team


In [16]:
print(mx.generation.generate("How are you feeling about life? I am feeling ", sampling_method="top-k"))

How are you feeling about life? I am feeling icky about this situation, but it's happening more and more to the point where I can't even think about it anymore. I think it's time I got to learn the "why" in life. A good person will go through it. It's not easy. It's difficult to live with. I have to. I have to. Even though one of the best things possible for me is to learn "why", I can't afford to stop thinking about it. I am trying hard. I am not at my best. I am doing just fine, just like I always want to do. I have a full body. I
